<a href="https://colab.research.google.com/github/AlperYildirim1/Solutions/blob/main/doktor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

# We have to check which Torch version for Xformers (2.3 -> 0.0.27)
from torch import __version__; from packaging.version import Version as V
xformers = "xformers==0.0.27" if V(__version__) < V("2.4.0") else "xformers"
!pip install --no-deps {xformers} trl peft accelerate bitsandbytes triton

In [2]:
from transformers import AutoTokenizer
from unsloth import FastLanguageModel
import torch

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


In [3]:
from huggingface_hub import login
login(token="")
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "Yujivus/lora_model4_test",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful
==((====))==  Unsloth 2024.8: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.27.post2. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/230 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/345 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/168M [00:00<?, ?B/s]

Unsloth 2024.8 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [4]:
prompt = """You are a conversational medical AI chatbot. Based on the provided input, respond appropriately by either diagnosing the symptoms and recommending the appropriate medical clinic or by answering questions about specific diseases, including prevention methods.

### Instruction:
Based on the input, either:
1. If provided with symptoms, list a range of possible most likely diagnoses. Recommend the appropriate medical clinic for further evaluation.
2. Provide detailed information about a specific disease if asked, including its symptoms, causes, and prevention methods.
3. Clearly state the name of the medical clinic to which you are directing the patient.

### Human_Question:
{input}

### AI_Response:
{output}

"""



In [16]:
FastLanguageModel.for_inference(model)
while True:
    user_input = input ()
    if user_input.lower() in ['exit', 'quit', 'q']:
        print("Görüşmek üzere!")
        break
    formatted_prompt = prompt.format(
        input=user_input,  # Example input
        output=""  # Leave this blank for generation
    )

    # Tokenize the formatted prompt
    inputs = tokenizer([formatted_prompt], return_tensors="pt").to("cuda")

    # Generate the output
    outputs = model.generate(**inputs, max_new_tokens=256, use_cache=True)

    # Decode and print the output
    decoded_outputs = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    print(decoded_outputs[0])


dear doctor, my child age is 4 yrs. from last 1 month, she is doing her motion for every 3-4 days with little hard kind of motion. everyday she is feeling like going for motion in her panty. but she is actually not going? plz help. her eating habits are normal and physically active
You are a conversational medical AI chatbot. Based on the provided input, respond appropriately by either diagnosing the symptoms and recommending the appropriate medical clinic or by answering questions about specific diseases, including prevention methods.

### Instruction:
Based on the input, either:
1. If provided with symptoms, list a range of possible most likely diagnoses. Recommend the appropriate medical clinic for further evaluation.
2. Provide detailed information about a specific disease if asked, including its symptoms, causes, and prevention methods.
3. Clearly state the name of the medical clinic to which you are directing the patient.

### Human_Question:
dear doctor, my child age is 4 yrs. f